# Description

This notebook will create the assets needed to run the rest of this demo.

# Boilerplate

## Dependencies

In [0]:
%pip install dbldatagen
%restart_python

## Parameters

In [0]:
dbutils.widgets.text("catalog_name", "", "00 - Catalog Name")
dbutils.widgets.text("schema_name", "", "01 - Schema Name")

In [0]:
params = dbutils.widgets.getAll()

for key, value in params.items():
  assert value != "", f"Parameter {key} is empty"

locals().update(params)

# Main

## Unity Catalog Assets

In [0]:
#spark.sql(f"CREATE CATALOG IF NOT EXISTS {catalog_name}");
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {catalog_name}.{schema_name}");
spark.sql(f"CREATE VOLUME IF NOT EXISTS {catalog_name}.{schema_name}.input_volume")

## Data Assets

Using [dbldatagen.datasets.multi_table_telephony_provider](https://databrickslabs.github.io/dbldatagen/public_docs/reference/api/dbldatagen.datasets.multi_table_telephony_provider.html#module-dbldatagen.datasets.multi_table_telephony_provider)

In [0]:
import dbldatagen as dg
import pyspark.sql.functions as F

### Parameters

In [0]:
UNIQUE_PLANS = 20
PLAN_MIN_VALUE = 100

UNIQUE_CUSTOMERS = 1000
CUSTOMER_MIN_VALUE = 1000
DEVICE_MIN_VALUE = 1000000000
SUBSCRIBER_NUM_MIN_VALUE = 1000000000

AVG_EVENTS_PER_CUSTOMER = 50

### Plan Data

In [0]:
shuffle_partitions_requested = 8
partitions_requested = 1
data_rows = UNIQUE_PLANS # we'll generate one row for each plan

plan_dataspec = (
    dg.DataGenerator(spark, rows=data_rows, partitions=partitions_requested)
    .withColumn("plan_id","int", minValue=PLAN_MIN_VALUE, uniqueValues=UNIQUE_PLANS)
    # use plan_id as root value
    .withColumn("plan_name", prefix="plan", baseColumn="plan_id")

    # note default step is 1 so you must specify a step for small number ranges,
    .withColumn("cost_per_mb", "decimal(5,3)", minValue=0.005, maxValue=0.050,
                step=0.005, random=True)
    .withColumn("cost_per_message", "decimal(5,3)", minValue=0.001, maxValue=0.02,
                step=0.001, random=True)
    .withColumn("cost_per_minute", "decimal(5,3)", minValue=0.001, maxValue=0.01,
                step=0.001, random=True)

    # we're modelling long distance and international prices simplistically -
    # each is a multiplier thats applied to base rate
    .withColumn("ld_multiplier", "decimal(5,3)", minValue=1.5, maxValue=3, step=0.05,
                random=True, distribution="normal", omit=True)
    .withColumn("ld_cost_per_minute", "decimal(5,3)",
                expr="cost_per_minute * ld_multiplier",
                baseColumns=['cost_per_minute', 'ld_multiplier'])
    .withColumn("intl_multiplier", "decimal(5,3)", minValue=2, maxValue=4, step=0.05,
                random=True,  distribution="normal", omit=True)
    .withColumn("intl_cost_per_minute", "decimal(5,3)",
                expr="cost_per_minute * intl_multiplier",
                baseColumns=['cost_per_minute', 'intl_multiplier'])
            )

df_plans = (
    plan_dataspec.build()
    .withColumn("plan_id", F.col("plan_id").cast("string"))
)

(
    df_plans
    .write.mode("overwrite")
    .saveAsTable(f"{catalog_name}.{schema_name}.plans")
)

In [0]:
%sql
SELECT
  *
FROM IDENTIFIER(:catalog_name || '.' || :schema_name || '.' || 'plans')
LIMIT 10

### Customers

In [0]:
shuffle_partitions_requested = 8
partitions_requested = 8
data_rows = UNIQUE_CUSTOMERS

customer_dataspec = (dg.DataGenerator(spark, rows=data_rows, partitions=partitions_requested)
            .withColumn("customer_id","decimal(10)", minValue=CUSTOMER_MIN_VALUE,
                        uniqueValues=UNIQUE_CUSTOMERS)
            .withColumn("customer_name", template=r"\\w \\w|\\w a. \\w")

            # use the following for a simple sequence
            #.withColumn("device_id","decimal(10)", minValue=DEVICE_MIN_VALUE,
            #              uniqueValues=UNIQUE_CUSTOMERS)

            .withColumn("device_id","decimal(10)",  minValue=DEVICE_MIN_VALUE,
                        baseColumn="customer_id", baseColumnType="hash")

            .withColumn("phone_number","decimal(10)",  minValue=SUBSCRIBER_NUM_MIN_VALUE,
                        baseColumn=["customer_id", "customer_name"], baseColumnType="hash")

            # for email, we'll just use the formatted phone number
            .withColumn("email","string",  format="subscriber_%s@myoperator.com",
                        baseColumn="phone_number")
            .withColumn("plan_id", "int", minValue=PLAN_MIN_VALUE, uniqueValues=UNIQUE_PLANS,
                        random=True)
            )

df_customers = (customer_dataspec.build()
                .dropDuplicates(["device_id"])
                .dropDuplicates(["phone_number"])
                .orderBy("customer_id")
                .withColumn("customer_id", F.col("customer_id").cast("string"))
                .withColumn("device_id", F.col("device_id").cast("string"))
                .withColumn("plan_id", F.col("plan_id").cast("string"))
               )

effective_customers = df_customers.count()

print(
  f"""revised customers : {df_customers.count()},
   |   unique customers: {df_customers.select(F.countDistinct('customer_id')).take(1)[0][0]},
   |   unique device ids: {df_customers.select(F.countDistinct('device_id')).take(1)[0][0]},
   |   unique phone numbers: {df_customers.select(F.countDistinct('phone_number')).take(1)[0][0]}""".strip()
     )

(
  df_customers
  .write.mode("overwrite")
  .saveAsTable(f"{catalog_name}.{schema_name}.customers")
)

In [0]:
%sql
SELECT
  *
FROM IDENTIFIER(:catalog_name || '.' || :schema_name || '.' || 'customers')
LIMIT 10

### Device Events

In [0]:
shuffle_partitions_requested = 8
partitions_requested = 8
NUM_DAYS=365
MB_100 = 100 * 1000 * 1000
K_1 = 1000
data_rows = AVG_EVENTS_PER_CUSTOMER * UNIQUE_CUSTOMERS * NUM_DAYS



# use random seed method of 'hash_fieldname' for better spread - default in later builds
events_dataspec = (dg.DataGenerator(spark, rows=data_rows, partitions=partitions_requested,
                   randomSeed=42, randomSeedMethod="hash_fieldname")
             # use same logic as per customers dataset to ensure matching keys
             # but make them random
            .withColumn("device_id_base","decimal(10)", minValue=CUSTOMER_MIN_VALUE,
                        uniqueValues=UNIQUE_CUSTOMERS,
                        random=True, omit=True)
            .withColumn("device_id","decimal(10)",  minValue=DEVICE_MIN_VALUE,
                        baseColumn="device_id_base", baseColumnType="hash")

            # use specific random seed to get better spread of values
            .withColumn("event_type","string",
                        values=[ "sms", "internet", "local call", "ld call", "intl call" ],
                        weights=[50, 50, 20, 10, 5 ], random=True)

            # use Gamma distribution for skew towards short calls
            .withColumn("base_minutes","decimal(7,2)",
                        minValue=1.0, maxValue=100.0, step=0.1,
                        distribution=dg.distributions.Gamma(shape=1.5, scale=2.0),
                        random=True, omit=True)

            # use Gamma distribution for skew towards short transfers
            .withColumn("base_bytes_transferred","decimal(12)",
                        minValue=K_1, maxValue=MB_100,
                        distribution=dg.distributions.Gamma(shape=0.75, scale=2.0),
                        random=True, omit=True)

            .withColumn("minutes", "decimal(7,2)",
                        baseColumn=["event_type", "base_minutes"],
                        expr= """
                              case when event_type in ("local call", "ld call", "intl call")
                                  then base_minutes
                                  else 0
                              end
                               """)
            .withColumn("bytes_transferred", "decimal(12)",
                        baseColumn=["event_type", "base_bytes_transferred"],
                        expr= """
                              case when event_type = "internet"
                                   then base_bytes_transferred
                                   else 0
                              end
                               """)
            
            .withColumn("event_second_shift", "decimal(10,2)",
                        minValue=0.0, maxValue=365*24*60*60,
                        random=True, omit=True)

            .withColumn("event_ts", "timestamp",
                        baseColumn=["event_second_shift"],
                        expr=f"""
                                timestampadd(SECOND, event_second_shift, current_date() - {NUM_DAYS})
                             """)
            )

df_events = (
    events_dataspec.build()
    .withColumn("device_id", F.col("device_id").cast("string"))
)

(
    df_events
    .write.mode("overwrite")
    .saveAsTable(f"{catalog_name}.{schema_name}.events")
)

In [0]:
%sql
SELECT
  *
FROM IDENTIFIER(:catalog_name || '.' || :schema_name || '.' || 'events')
LIMIT 10

### Customer Demographics

In [0]:
shuffle_partitions_requested = 8
partitions_requested = 8
data_rows = UNIQUE_CUSTOMERS

customer_demographics_dataspec = (
  dg.DataGenerator(spark, rows=data_rows, partitions=partitions_requested)
  .withColumn("customer_id","decimal(10)", minValue=CUSTOMER_MIN_VALUE,
              uniqueValues=UNIQUE_CUSTOMERS)

  .withColumn("days_lived","decimal(7,2)",
              minValue=365*15, maxValue=365*50, step=0.1,
              distribution= dg.distributions.Beta(alpha=12, beta=1.5),
              random=True, omit=True)
  
  .withColumn("birth_date", "date",
                        baseColumn=["days_lived"],
                        expr=f"""
                                timestampadd(DAY, -1 * days_lived, current_date())
                             """)

  .withColumn("days_as_customer","decimal(7,2)",
              minValue=1.0, maxValue=365*10, step=0.1,
              distribution= dg.distributions.Beta(alpha=12, beta=1.5),
              random=True, omit=True)

  .withColumn("customer_since", "date",
                        baseColumn=["days_as_customer"],
                        expr=f"""
                                timestampadd(DAY, -1 * days_as_customer, current_date())
                             """)

  .withColumn("location","string",
              values=["Aguascalientes", "Baja California", "Baja California Sur", "Campeche",
                      "Chiapas", "Chihuahua", "Coahuila", "Colima", "Durango", "Guanajuato",
                      "Guerrero", "Hidalgo", "Jalisco", "México", "Michoacán", "Morelos",
                      "Nayarit", "Nuevo León", "Oaxaca", "Puebla", "Querétaro", "Quintana Roo",
                      "San Luis Potosí", "Sinaloa", "Sonora", "Tabasco", "Tamaulipas", "Tlaxcala",
                      "Veracruz", "Yucatán", "Zacatecas", "Ciudad de México"],
              weights=[14, 37, 7, 9, 55, 38, 31, 7, 18, 62,
                        35, 31, 83, 174, 48, 19, 13, 55, 41,
                        65, 22, 18, 28, 30, 29, 22, 37, 13,
                        81, 22, 16,92], random=True)
  )

df_customer_demographics = (
  customer_demographics_dataspec.build()
  .dropDuplicates(["customer_id"])
  .withColumn("customer_id", F.col("customer_id").cast("string"))
)


(
  df_customer_demographics
  .write.mode("overwrite")
  .saveAsTable(f"{catalog_name}.{schema_name}.customer_demographics")
)

In [0]:
%sql
SELECT * FROM IDENTIFIER(:catalog_name || '.' || :schema_name || '.' || 'customer_demographics')
LIMIT 10

### Customer Plan Changes

TODO: WHEN A CUSTOMER CHANGES FROM ONE PLAN TO ANOTHER

### Add Primary Keys & Foreign Keys

In [0]:
%sql
ALTER TABLE IDENTIFIER(:catalog_name || '.' || :schema_name || '.' || 'plans')
  ALTER COLUMN plan_id SET NOT NULL;

ALTER TABLE IDENTIFIER(:catalog_name || '.' || :schema_name || '.' || 'customers')
  ALTER COLUMN customer_id SET NOT NULL;

ALTER TABLE IDENTIFIER(:catalog_name || '.' || :schema_name || '.' || 'customer_demographics')
  ALTER COLUMN customer_id SET NOT NULL;

ALTER TABLE IDENTIFIER(:catalog_name || '.' || :schema_name || '.' || 'customer_demographics')
  ADD CONSTRAINT customer_demographics_pk PRIMARY KEY (customer_id);

In [0]:
%sql
ALTER TABLE IDENTIFIER(:catalog_name || '.' || :schema_name || '.' || 'plans')
  ADD CONSTRAINT plans_pk PRIMARY KEY (plan_id);

ALTER TABLE IDENTIFIER(:catalog_name || '.' || :schema_name || '.' || 'customers')
  ADD CONSTRAINT customers_pk PRIMARY KEY (customer_id);

/*
ALTER TABLE IDENTIFIER(:catalog_name || '.' || :schema_name || '.' || 'customers')
  ADD CONSTRAINT customers_plans_fk
    FOREIGN KEY(plan) REFERENCES IDENTIFIER(:catalog_name || '.' || :schema_name || '.' || 'plans');
*/

In [0]:
spark.sql(f"""
ALTER TABLE {catalog_name}.{schema_name}.customers
  ADD CONSTRAINT customers_plans_fk
    FOREIGN KEY(plan_id) REFERENCES {catalog_name}.{schema_name}.plans;""")

spark.sql(f"""
ALTER TABLE {catalog_name}.{schema_name}.customer_demographics
  ADD CONSTRAINT customer_demographic_customers_fk
    FOREIGN KEY(customer_id) REFERENCES {catalog_name}.{schema_name}.customers;""")